using Revise

In [1]:
using TopOpt, LinearAlgebra, StatsFuns
using StatsFuns: logsumexp

E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force
rmin = 3.0

3.0

### Define the problem

In [2]:
problem = PointLoadCantilever(Val{:Linear}, (160, 40), (1.0, 1.0), E, v, f)

### Parameter settings

In [3]:
V = 0.5 # volume fraction
xmin = 0.0001 # minimum density
steps = 40 # maximum number of penalty steps, delta_p0 = 0.1

40

### Continuation SIMP

In [4]:
x0 = fill(1.0, 160 * 40) # initial design
x = copy(x0)
for p in [1.0, 2.0, 3.0]
    global penalty, stress, filter, result, stress, x
    penalty = TopOpt.PowerPenalty(p)
    solver = FEASolver(Direct, problem; xmin=xmin, penalty=penalty)
    stress = TopOpt.MicroVonMisesStress(solver)
    filter = DensityFilter(solver; rmin=rmin)
    volfrac = TopOpt.Volume(problem, solver)

    obj = x -> volfrac(filter(x)) - V
    thr = 10 # stress threshold
    constr = x -> begin
        s = stress(filter(x))
        vcat((s .- thr) / 100, logsumexp(s) - log(length(s)) - thr)
    end
    alg = PercivalAlg()
    options = PercivalOptions()
    optimizer = Optimizer(obj, constr, x, alg; options=options)
    simp = SIMP(optimizer, solver, p)
    result = simp(x)
    x = result.topology
end

maximum(stress(filter(x0))) # 0.51
maximum(stress(filter(x))) # 10.01

[ Info:   iter        fx    normgp    normcx         μ     normy    sumc     inner_status        iter_type  
[ Info:      0   5.0e-01   8.0e+01   1.3e+01   1.0e+01   8.0e+01       5
[ Info:      1   5.0e-01   8.0e+01   1.3e+01   1.0e+02   8.0e+01       9      first_order         update_μ
[ Info:      2   5.0e-01   8.0e+01   8.0e-01   1.0e+03   8.0e+01      21      first_order         update_μ
[ Info:      3   5.0e-01   1.3e-02   8.0e-02   1.0e+03   1.9e-05      32      first_order         update_y
[ Info:      4  -4.3e-01   6.4e-02   5.9e-05   1.0e+03   5.9e-02     530         max_eval         update_y
[ Info:      5  -4.3e-01   6.4e-02   5.9e-05   1.0e+04   5.9e-02     534         max_eval         update_μ
[ Info:      6  -4.3e-01   7.2e-01   5.9e-05   1.0e+04   6.5e-01     538         max_eval         update_y
[ Info:      7  -4.3e-01   1.4e+00   5.9e-05   1.0e+04   1.2e+00     542         max_eval         update_y
[ Info:      8  -4.3e-01   1.4e+00   5.9e-05   1.0e+05   1.2e+00     

17.771315897497438

### (Optional) Visualize the result using Makie.jl
Need to run `using Pkg; Pkg.add(Makie)` first
```julia
using Makie
using TopOpt.TopOptProblems.Visualization: visualize
fig = visualize(
   problem; topology = penalty.(filter(result.topology)), default_exagg_scale = 0.07,
   scale_range = 10.0, vector_linewidth = 3, vector_arrowsize = 0.5,
)
Makie.display(fig)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*